#Waymo Open Dataset 

- Website: https://waymo.com/open
- GitHub: https://github.com/waymo-research/waymo-open-dataset


This notebook demonstratea how to read the labels, and to extract panoptic labels with consistent instance ID tracks for any number of frames but they are all converted to match the kitti step dataset format.

This notebook mainly relies on the Waymo Open Dataset library for processing.



# Package Installation

In [ ]:
!pip3 install waymo-open-dataset-tf-2-11-0==1.5.1

In [ ]:
#@markdown ## (Optional) Install a minimal version of deeplab2 for the WOD.
# Please ignore this cell if you already have deeplab2 installed

# This shell script will download and install only those deeplab2 modules which
# are used by the WOD.
# They are used here https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/bazel/deeplab2.BUILD
!wget https://raw.githubusercontent.com/waymo-research/waymo-open-dataset/master/src/waymo_open_dataset/pip_pkg_scripts/install_deeplab2.sh -O - | bash

# Refer to the instructions on how intall the entire deeplab2 if you need other
# deeplab2 modules as well.
# https://github.com/google-research/deeplab2/blob/main/g3doc/setup/installation.md

In [ ]:
!pip install --ignore-installed Pillow==9.0.0

# Imports and global definitions

The tfrecords of the waymo dataset are extracted from the official data saved on google cloud. 

To extract them, you have to have an account on the waymo dataset

In [ ]:
!gcloud auth login

In [ ]:
txt = open("/content/waymo.txt", "r")
files = txt.read().split("\n")
files = files[61:71]
# files = files[0]

In [ ]:
len(files)

In [ ]:
for file in files:
  !gsutil cp gs://$file /content/training

In [ ]:
import os 
folder_path = "/content/training" 
FILE_NAMES  = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
FILE_NAMES 

In [ ]:
import os
from typing import Any, Dict, Iterator, List, Optional, Sequence, Tuple
import immutabledict
import matplotlib.pyplot as plt
import tensorflow as tf
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd

if not tf.executing_eagerly():
  tf.compat.v1.enable_eager_execution()

from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset import v2
from waymo_open_dataset.protos import camera_segmentation_metrics_pb2 as metrics_pb2
from waymo_open_dataset.protos import camera_segmentation_submission_pb2 as submission_pb2
from waymo_open_dataset.wdl_limited.camera_segmentation import camera_segmentation_metrics
from waymo_open_dataset.utils import camera_segmentation_utils
from PIL import Image
from waymo_open_dataset import dataset_pb2

# Process Waymo data 

We process the Waymo data to Kitti format which is based on the Coco format. 

The channels of the image are formatted as follows:
```
Red Channel = semantic_id
Green Channel = instance_id // 256
Blue Channel  = instance % 256
```

Each dataset has different labels ids so these ids are mapped to each other to be matched together. To ensure tracking, the glbal mapping ids were calculated using the Waymo library. The global ids are consistant throughout each video sequence and throughout the 5 different cameras.





In [ ]:
# define label maps for waymmo and kitti 

waymo_label = ["void","TYPE_EGO_VEHICLE", "car", "truck", "bus","large_vehicles","bicycle","motorcycle","trailer", "person","cyclist","motorcyclist","bird", "animal","cone_pole", "pole", "person_obj", "traffic_sign", "traffic_light", "building", "road", "road1","road2", "sidewalk", "vegetation", "sky", "ground", "TYPE_DYNAMIC","TYPE_STATIC"]
waymo_label_map = {label: x for x, label in enumerate(waymo_label)}
kitti_label= ["road", "sidewalk", "building", "wall", "fence", "pole", "traffic_light", "traffic_sign", "vegetation","terrain", "sky", "person", "rider", "car", "truck", "bus", "train", "motorcycle", "bicycle", "void"  ]
kitti_label_map = {label: x for x, label in enumerate(kitti_label)}
kitti_label_map["void"] = 255

In [ ]:
count = 0  ## change this to the last saved count 
for FILE_NAME in FILE_NAMES:
  dataset = tf.data.TFRecordDataset(FILE_NAME, compression_type='')
  frames_with_seg = []
  for data in dataset:
    sequence_id = None
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    # Save frames which contain CameraSegmentationLabel messages. We assume that
    # if the first image has segmentation labels, all images in this frame will.
    if frame.images[0].camera_segmentation_label.panoptic_label:
      
      if sequence_id is None:
        sequence_id = frame.images[0].camera_segmentation_label.sequence_id
      for index, image in enumerate(frame.images):
         
          if(index == 0):
            count += 1
            string_count = '{:05d}'.format(count)
          
          #get RGB images
          
          im = tf.image.decode_jpeg(image.image)
          
          image_name = "waymo_out/"+ str(sequence_id[-6:]) + "_" + str(image.name) + string_count + "_leftImg8bit" + ".png"
          
          tf.keras.preprocessing.image.save_img(image_name, im)

         #get global id for tracking instances 
          global_id_mapping = camera_segmentation_utils._remap_global_ids(
              [image.camera_segmentation_label], remap_to_sequential=False)
          if global_id_mapping:
            max_instance_id = max(
                [max([global_id for _, global_id in mapping.items()])
                for _, mapping in global_id_mapping.items()])
      
          #get semantic and instances labels for processing 
          panoptic_labels = camera_segmentation_utils.decode_single_panoptic_label_from_proto(image.camera_segmentation_label)
          semantic_label, instance_label = camera_segmentation_utils.decode_semantic_and_instance_labels_from_panoptic_label(panoptic_labels, image.camera_segmentation_label.panoptic_label_divisor)
          
          instance_label_copy = np.copy(instance_label)

          # use waymo library functions to map the instance labels to global labels
          if True:
            if isinstance(image.camera_segmentation_label, v2.CameraSegmentationLabelComponent):
              mapping_iter = camera_segmentation_utils._iterate_over_mapping(label)
            elif isinstance(image.camera_segmentation_label, dataset_pb2.CameraSegmentationLabel):
              mapping_iter = image.camera_segmentation_label.instance_id_to_global_id_mapping
            else:
              raise ValueError('Input label format not supported.')
            for mapping in mapping_iter:
              instance_mask = (instance_label == mapping.local_instance_id)
              instance_label_copy[instance_mask] = global_id_mapping[image.camera_segmentation_label.sequence_id][
                  mapping.global_instance_id]
          if np.amax(instance_label) >= image.camera_segmentation_label.panoptic_label_divisor:
            raise ValueError('A panoptic_label_divisor of '
                            f'{image.camera_segmentation_label.panoptic_label_divisor} is requested, but the '
                            'maximum instance id exceeds this.')
          
          instance_label_copy = instance_label_copy + 1
          
          # use the label maps to map waymo labels into kitti labels
          inverted_dict= {v: k for k, v in waymo_label_map.items()}
          
          new_semantic_labels = semantic_label.copy()
          for idx, label in np.ndenumerate(semantic_label):
            key = inverted_dict.get(label)
            if key == "person_obj":
                key = "person"
            if key == "road1" or key == "road2":
                key = "road"
            if key in kitti_label_map.keys():
              new_semantic_labels[idx] = kitti_label_map[key]
            else: 
              new_semantic_labels[idx] = kitti_label_map["void"]
         
         #format the images to be similar to kitti and concantenate the channels

          R = new_semantic_labels
          G = instance_label_copy // 256
          B = instance_label_copy % 256
          panoptic_label_rgb = np.dstack((R,G,B))
          im = Image.fromarray((panoptic_label_rgb).astype(np.uint8))
              
          image_name = "waymo_out/" + str(sequence_id[-6:]) + "_" + str(image.name) + string_count + "_panoptic" + ".png"
          im.save(image_name)
    

In [ ]:
### uncomment to visualise images 


# panoptic_label_rgb = camera_segmentation_utils.panoptic_label_to_rgb(semantic_label, instance_label)
    # plt.figure(figsize=(64, 60))
    # plt.imshow(panoptic_label_rgb)
    # plt.grid(False)
    # plt.axis('off')
    # plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#save folder in google drive

!cp -r "/content/waymo_out7" "/content/drive/MyDrive/waymo_new/"